# Modelo NLP - Twitter

In [ ]:
# Installs
#!pip3 install scikit-learn
#!pip3 install -U spacy
#!python3 -m spacy download es
#!python3 -m spacy download es_core_news_sm

In [ ]:
#!pip install spacy

In [ ]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download es_core_news_sm

In [1]:
import pandas as pd
from spacy import displacy
import spacy
import re
import mysql.connector as conn

In [2]:
from spacy.lang.es.stop_words import STOP_WORDS

#Crear pdf report

from io import BytesIO
from reportlab.pdfgen import canvas
from django.http import HttpResponse

#Automatizar email

import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [20]:
import spacy
from spacy import displacy

# Cargo Modelo NLP

In [64]:
import pickle

In [65]:
clf = pickle.load(open(filename, 'rb'))

In [67]:
clf.predict(['Estas muy guapa'])

array([1])

## Creo base de datos necesarias

### Creando SQL Database

In [68]:
db=conn.connect(host='localhost', user='root', passwd='123456', database='hate_blocker')

cursor=db.cursor()

cursor.execute('drop table if exists hate_blocker')

### Creando Tabla: Train Model

El objetivo de este tabla es guardar los siguientes tweets:

- Tweets correctos identificados como sentiment 1.

- Tweets incorrectos identificados como sentiment 0.

- Tweets correctos identificados como sentiment 0.

cursor.execute('drop table if exists train_model')

In [ ]:
tabla= '''
        create table train_model(
            
           tweet_id bigint ,      
           user bigint,
           content varchar(500),
           date varchar(500),
           lang varchar(500),
           sentiment bigint
            
        );
'''

cursor.execute (tabla)

### Creando Tabla: Listado Bullies

cursor.execute('drop table if exists listado_bullies')

In [ ]:
tabla= '''
        create table listado_bullies(
            
           
           tweet_id bigint,
           date varchar(500),      
           account varchar(500),
           tweet_text varchar(500),
           URL_account varchar(500),
           URL_tweet varchar(500)
            
        );
'''


cursor.execute (tabla)

# Funcion PDF

In [46]:
def drawMyRuler (pdf): #Plantilla
    
    pdf.drawString(100, 810, 'x100')
    pdf.drawString(200, 810, 'x200')
    pdf.drawString(300, 810, 'x300')
    pdf.drawString(400, 810, 'x400')
    pdf.drawString(500, 810, 'x500')
    
    pdf.drawString(10, 100, 'y100')
    pdf.drawString(10, 200, 'y200')
    pdf.drawString(10, 300, 'y300')
    pdf.drawString(10, 400, 'y400')
    pdf.drawString(10, 500, 'y500')
    pdf.drawString(10, 600, 'y600')
    pdf.drawString(10, 700, 'y700')
    pdf.drawString(10, 800, 'y800')

# Funcion PDF

In [47]:
def pdf(usuario, texto):
            
        #name and create pdf

        c = canvas.Canvas('hate_.pdf')
        
        #set background color (this color is yellow I do not recommend)

        c.setFillColorRGB(0,0,0)

        #Select font and font size

        c.setFont('Helvetica', 12)

        #create two text strings and set there position on the page

        c.drawString(5, 660, 'Hemos bloqueado la cuenta' + usuario + 'por el siguiente tweet:' + texto)

        #show page and save it

        c.showPage()
        c.save()
          

# Función-Modelo

In [59]:
import tweepy

In [60]:
# 4 authentication chains

consumer_key='Qbxaw0OrPeZSFljKEp6SHI19M'
consumer_secret='kva4UfB6cK2Z6P70MGV8LAM0MMOu3SAHjuUl5t7bWxpegkCHGl'
access_key='902474996-eLIpuPP1uqYZoTWtprZuiL2FmehUURAZHT8ZBReU'
access_secret='CkVPZlMz5Lk4EMYDZzmNM2zJQHGE0ze6CA0auHqnYtEhU'

# authorize twitter, initialize tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [61]:

def insert(sentiment):
    
    insert_query="insert into {} ({}) values {};"\
                        .format('train_model', ','.join(['tweet_id', 'user', 'content', 'date', 'lang', 'sentiment']), tuple([tweet.id_str, i.id,tweet.text, str(tweet.created_at), 'es', sentiment]))


    cursor.execute(insert_query)

    db.commit()

    return

In [62]:
def bullies_to_sql():
    
    insert_query="insert into {} ({}) values {};"\
                        .format('listado_bullies', ','.join(['tweet_id','date', 'account', 'tweet_text', 'URL_account','URL_tweet']), 
                                tuple([tweet.id_str, str(tweet.created_at), i.screen_name,tweet.text, 'https://twitter.com/' + i.screen_name, 'https://twitter.com/' + i.screen_name + '/status/' + tweet.id_str]))


    cursor.execute(insert_query)

    db.commit()

    return

In [69]:
users = []
users_url = []
tweets_url = []

usuario = []
texto = []


mentions = api.mentions_timeline(tweet_mode= 'extented')

for tweet in mentions: # Entran los tweets
    
    
    users.append(tweet.user)

    for i in users:
        
        print(' ')
    
    valoracion = clf.predict([tweet.text]) 
    
    if valoracion == 1: # Es positivo el tweet?
        
        print ('No bloqueo al usuario. Guardado para entrenar.')
        
        print (' ')
        
        insert(1) # Mete en sql los tweets con etiqueta 1 para entrenar.

        
    elif valoracion == 0: # Es negativo el tweet?
        
        print('Voy a bloquear a ', tweet.user.screen_name, 'por el siguiente tweet:', tweet.text )
               
        verif = input ('¿Deseas bloquear a este usuario?') # Estas seguro que quieres bloquear?
        
        print (verif)
        
        if verif == 'Si': # Si
            
            api.create_block(tweet.user.screen_name)
            
            insert(0) # Metelo en sql los tweets con etiqueta 0 para entrenar.
            
            bullies_to_sql() # Metelo en el listado de bullies para enseñarlo al cliente.
            
            print('Bloqueado. Guardado para entrenar.')
            
            print('Guardado para enviar a cliente. ')
            
            usuario.append(i.name)
            
            texto.append(tweet.text)
            
            #pdf(usuario, texto)
            
            
            
        elif verif == 'No': # No
            
            insert(1) # Mete en sql los tweets con etiqueta 1 para entrenar.
            
            print ('Usuario no bloqueado. Guardado para entrenar')

 
Voy a bloquear a  CreedMoreno por el siguiente tweet: @biwenger @xinxinricardo @CarlosLarios96


KeyboardInterrupt: Interrupted by user